In [50]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [51]:
import nltk

# # This will download stopwords if not already present
# nltk.download('stopwords') wordNet

nltk.download('stopwords', download_dir='F:/Programming_Practise/MLOPs_Project1/notebooks')
nltk.download('wordnet', download_dir='F:/Programming_Practise/MLOPs_Project1/notebooks')
import nltk.data
nltk.data.path.append('F:/Programming_Practise/MLOPs_Project1/notebooks')
# nltk.data.path.append('F:/Programming_Practise/MLOPs_Project1/notebooks')



[nltk_data] Downloading package stopwords to
[nltk_data]     F:/Programming_Practise/MLOPs_Project1/notebooks...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     F:/Programming_Practise/MLOPs_Project1/notebooks...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
df=pd.read_csv("IMDB.csv")
df=df.sample(500)
df.to_csv("data.csv",index=False)
df.head()

,review,sentiment
388,Ostensibly a story about the young child of Ji...,positive
449,Currently playing at the 2007 German Film Fest...,positive
706,I am from the Dallas/Fort Worth area and lived...,negative
596,Emma is a horribly flawed film based on Jane A...,positive
844,"Repugnant Bronson thriller. Unfortunately, it'...",negative


In [53]:
# Data Preprocessing

# Define Text preprocessing functions
def lemmatization(text):
    """ Lemmatize the text. """
    lemmatizer=WordNetLemmatizer()
    text=text.split()
    text=[lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """ Remove stopswords from the text """
    stop_words= set(stopwords.words("english"))
    text=[word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """ Remove numbers from the text """
    text= "".join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """ Convert text to lower case """
    text= text.split()
    text=[word.lower() for word in text]
    return " ".join(text)

def removing_punctations(text):
    """ Removing puncatations from the text """
    text= re.sub('[%s]' % re.escape(string.punctuation),' ', text)
    text=text.replace(':','')
    text=re.sub('\s+',' ',text).strip()
    return text

def removing_urls(text):
    """ Remove URLs from the text. """
    url_pattern=re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)

def normalize_text(df):
    """ Normalize the text data."""
    try: 
        df["review"]= df["review"].apply(lower_case)
        df["review"]= df["review"].apply(remove_stop_words)
        df["review"]= df["review"].apply(removing_numbers)
        df["review"]= df["review"].apply(removing_punctations)
        df["review"]= df["review"].apply(removing_urls)
        df["review"]= df["review"].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during text normalization: {e}")
        raise

In [54]:
df = normalize_text(df)
df.head()

,review,sentiment
388,ostensibly story young child jimmy stewart dor...,positive
449,currently playing german film festival austral...,positive
706,dallas fort worth area lived arlington year mo...,negative
596,emma horribly flawed film based jane austen cl...,positive
844,repugnant bronson thriller unfortunately techn...,negative


In [55]:
df['sentiment'].value_counts()

sentiment
positive    250
negative    250
Name: count, dtype: int64

In [56]:
x=df['sentiment'].isin(['positive','negative'])
df=df[x]

In [57]:
df['sentiment']=df['sentiment'].map({'positive':1,'negative':0})
df.head()

,review,sentiment
388,ostensibly story young child jimmy stewart dor...,1
449,currently playing german film festival austral...,1
706,dallas fort worth area lived arlington year mo...,0
596,emma horribly flawed film based jane austen cl...,1
844,repugnant bronson thriller unfortunately techn...,0


In [58]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [59]:
vectorizer=CountVectorizer(max_features=75)
X=vectorizer.fit_transform(df['review'])
y=df['sentiment']

In [60]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,random_state=42)

In [61]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/NitinNandeshwar/MLOPs_Project1.mlflow')
dagshub.init(repo_owner='NitinNandeshwar',repo_name='MLOPs_Project1',mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Initialized MLflow to track repo "NitinNandeshwar/MLOPs_Project1"

2025-06-28 16:22:42,587 - INFO - Initialized MLflow to track repo "NitinNandeshwar/MLOPs_Project1" 


Repository NitinNandeshwar/MLOPs_Project1 initialized!

2025-06-28 16:22:42,589 - INFO - Repository NitinNandeshwar/MLOPs_Project1 initialized! 


<Experiment: artifact_location='mlflow-artifacts:/62e8187d480a4b5e812534a8cf4f655d', creation_time=1751122486198, experiment_id='0', last_update_time=1751122486198, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [62]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score

# Configure Logging
logging.basicConfig(level=logging.INFO,format="%(asctime)s - %(levelname)s - %(message)s ")

logging.info("Starting MLflow run ...")

with mlflow.start_run():
    start_time=time.time()

    try:
        logging.info("Logging preprocessing parameters ...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features",75)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initsializing Logistic Regression model ...")
        model= LogisticRegression(max_iter=1000) # Increase max_iter to prevent non-convergences issues

        logging.info("Fitting the model ...")
        model.fit(X_train,y_train)
        logging.info("Model training complete.")

        logging.info("Logging model paramters...")
        mlflow.log_param("model","Logistic Regression")

        logging.info("Making predictions ...")
        y_pred= model.predict(X_test)

        logging.info("Calculating evaluation metrics ...")
        accuracy=accuracy_score(y_test,y_pred)
        precision=precision_score(y_test,y_pred)
        recall=recall_score(y_test,y_pred)
        f1=f1_score(y_test,y_pred)

        logging.info("Logging evaluation metrics ...")
        mlflow.log_metric("accuracy",accuracy)
        mlflow.log_metric("precision",precision)
        mlflow.log_metric("recall",recall)
        mlflow.log_metric("f1_score",f1)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")


    except Exception as e:
        logging.error(f"An error occurred : {e}",exc_info=True)

2025-06-28 16:22:42,915 - INFO - Starting MLflow run ... 
2025-06-28 16:22:43,196 - INFO - Logging preprocessing parameters ... 
2025-06-28 16:22:44,034 - INFO - Initsializing Logistic Regression model ... 
2025-06-28 16:22:44,035 - INFO - Fitting the model ... 
2025-06-28 16:22:44,043 - INFO - Model training complete. 
2025-06-28 16:22:44,043 - INFO - Logging model paramters... 
2025-06-28 16:22:44,320 - INFO - Making predictions ... 
2025-06-28 16:22:44,321 - INFO - Calculating evaluation metrics ... 
2025-06-28 16:22:44,326 - INFO - Logging evaluation metrics ... 
2025-06-28 16:22:45,551 - INFO - Model training and logging completed in 2.35 seconds. 
2025-06-28 16:22:45,552 - INFO - Accuracy: 0.62 
2025-06-28 16:22:45,554 - INFO - Precision: 0.6428571428571429 
2025-06-28 16:22:45,554 - INFO - Recall: 0.6666666666666666 
2025-06-28 16:22:45,555 - INFO - F1 Score: 0.6545454545454545 
2025/06/28 16:22:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run unique-crane-930 at: ht